In [1]:
import numpy as np
import itertools as itr
import matplotlib.pyplot as plt

from torch import Tensor, manual_seed, is_tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS, Adam
from torch.utils.data import Dataset, DataLoader

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute, transpile, assemble, IBMQ
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.opflow import AerPauliExpectation
from qiskit.circuit import Parameter,ParameterVector
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import EfficientSU2, RealAmplitudes, ZZFeatureMap
from qiskit.opflow.gradients import Gradient, NaturalGradient, QFI, Hessian
#from sklearn.preprocessing import OneHotEncoder
from qiskit.algorithms.optimizers import SPSA, GradientDescent, QNSPSA, ADAM
#from qiskit.tools.monitor import job_monitor


from qiskit_machine_learning.runtime import TorchRuntimeClient, TorchRuntimeResult
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import CircuitQNN, TwoLayerQNN
from qiskit_machine_learning.circuit.library import RawFeatureVector

COUNTER = 0
DIR_val_train = "Encode_data/amp_enc_data_set_trainning_values.csv"
DIR_cls_train = "Encode_data/amp_enc_data_set_trainning_classes.csv"

DIR_val_test = "Encode_data/amp_enc_data_set_test_values.csv"
DIR_cls_test = "Encode_data/amp_enc_data_set_test_classes.csv"

#IBMQ.save_account("07916ec631273d08971f67f1267677801b440be43215767571a165abe0ac621415a17cc5a357e2e2fffa511a6fd3748eb0c46d35ca79d8752b97788fd71f390a", overwrite=True)

IBMQ.load_account()

provider = IBMQ.get_provider(hub="ibm-q")  # replace by your runtime provider
backend = provider.get_backend("ibmq_manila")  # select a backend that supports the runtime

In [14]:
def get_encode(file):
    """
     Funcao responsavel pelo encoding (amplitude)
        Input :: 
        ### file : File dir
        Output :: 
        #### data_enc : Valores para o encode (numpy array)
    """
    return np.genfromtxt(file, delimiter=";")

train_data = get_encode(DIR_val_train)
train_data_T = Tensor(train_data)
train_labels = np.genfromtxt(DIR_cls_train, delimiter=";")
train_labels_T = Tensor(train_labels).reshape(len(train_labels),1)

# Test data points
test_data = get_encode(DIR_val_test)
test_data_T = Tensor(test_data)
test_labels = np.genfromtxt(DIR_cls_test, delimiter=";")
test_labels_T = Tensor(test_labels).reshape(len(test_labels),1)

In [3]:
# Create custom torch dataset class
class TorchDataset(Dataset):
    """Map-style dataset"""

    def __init__(self, X, y):
        self.X = Tensor(X)
        self.y = Tensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        import torch

        if torch.is_tensor(idx):
            idx = idx.tolist()

        X_i = self.X[idx]
        y_i = self.y[idx]

        # important: the dataset item must be returned as data,target
        return X_i, y_i



In [16]:
train_set = TorchDataset(train_data_T, train_labels_T)
train_loader = DataLoader(train_set, batch_size=1, shuffle=False)
test_set = TorchDataset(test_data_T, test_labels_T)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)

In [23]:
n_qubits = 4

encoding = RawFeatureVector(16)

ansatz = EfficientSU2(n_qubits, entanglement='full', reps=4, 
                        insert_barriers=True, name="U(\u03B8)", 
                        parameter_prefix="\u03B8")

qnn1 = TwoLayerQNN(n_qubits, 
                   encoding, 
                   ansatz)
qnn1.circuit.draw()

qc = encoding.compose(ansatz)

qnn2 = parity = lambda x: "{:b}".format(x).count("1") % 2
output_shape = 2  
qnn6 = CircuitQNN(qc,
                    [],
                    qc.parameters,
                    sparse=False,
                    interpret=parity,
                    output_shape=output_shape)

initial_parameters = np.random.random(ansatz.num_parameters)
model1 = TorchConnector(qnn1, initial_parameters)
model2 = TorchConnector(qnn2, initial_parameters)

optimizer1 = Adam(model1.parameters(), lr=0.1)
loss_func1 = MSELoss(reduction="sum")

optimizer2 = Adam(model1.parameters(), lr=0.1)
loss_func2 = CrossEntropyLoss(reduction="sum")

Cannot compute gradient operator! Continuing without gradients!


In [8]:
torch_runtime_client = TorchRuntimeClient(
    provider=provider,
    model=model1,
    optimizer=optimizer1,
    loss_func=loss_func1,
    epochs=5,
    backend=backend,
)

torch_runtime_client3 = TorchRuntimeClient(
    provider=provider,
    model=model2,
    optimizer=optimizer2,
    loss_func=loss_func2,
    epochs=100,
    backend=backend,
)

In [9]:
fit_result = torch_runtime_client.fit(train_loader=train_loader)

In [10]:
# In this example, we use the same data loader for the prediction as well
predict_result = torch_runtime_client.predict(data_loader=train_loader)

In [26]:
def score_func(out, target):
    from numpy import sign
    score = 0
    print("out: ", out, type(out))
    print("target: ", target)
    if sign(out.item()) == target.item():
        score = 1
    return score

In [12]:
# data_loader = train_loader

score_result = torch_runtime_client.score(data_loader=train_loader, score_func=score_func)

In [13]:
print("id: ", score_result.job_id)
print("execution time: ", score_result.execution_time)
print("score: ", score_result.score)

id:  ca8b31s7krikd53s3aig
execution time:  551.3717706203461
score:  0.55


In [17]:
## data_laoder = test_loader

score_result = torch_runtime_client.score(data_loader=test_loader, score_func=score_func)

In [19]:
print("id: ", score_result.job_id)
print("execution time: ", score_result.execution_time)
print("score: ", score_result.score)

id:  ca8bfslmoms9e8gui290
execution time:  165.09437680244446
score:  0.5416666666666666


In [24]:
torch_runtime_client2 = TorchRuntimeClient(
    provider=provider,
    model=model1,
    optimizer=optimizer2,
    loss_func=loss_func2,
    epochs=50,
    backend=backend,
)

In [29]:
fit_result2 = torch_runtime_client2.fit(train_loader=train_loader, val_loader=test_loader)

In [30]:
predict_result = torch_runtime_client2.predict(data_loader=test_loader)

In [35]:
print(type(fit_result2.model_state_dict))

<class 'collections.OrderedDict'>


In [41]:
fit_result2.model_state_dict
fit_result2.val_history

[{'epoch': 0,
  'loss': 0.0,
  'forward_time': 6.59633994102478,
  'backward_time': 0,
  'epoch_time': 6.59633994102478},
 {'epoch': 1,
  'loss': 0.0,
  'forward_time': 6.885428428649902,
  'backward_time': 0,
  'epoch_time': 6.885428428649902},
 {'epoch': 2,
  'loss': 0.0,
  'forward_time': 6.381465196609497,
  'backward_time': 0,
  'epoch_time': 6.381465196609497},
 {'epoch': 3,
  'loss': 0.0,
  'forward_time': 6.3744072914123535,
  'backward_time': 0,
  'epoch_time': 6.3744072914123535},
 {'epoch': 4,
  'loss': 0.0,
  'forward_time': 6.380313158035278,
  'backward_time': 0,
  'epoch_time': 6.380313158035278}]

In [38]:
opt

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [40]:
print(predict_result.val_history)

None
